In [1]:
import os
import copy
from math import ceil

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [8]:
class MelodyIndexToChordSequenceDataset(Dataset):
    """
    This dataset loads the training data linking a 64 * 1
    melody index input to a 64 * 12 multi-hot chord sequence output
    """
    
    def __init__(self, data_dir, transpose=None, strip_rest_edge_measures=False):
        """
        Args:
            data_dir (string):
                directory where .txt files of training data are stored
            transpose (list[int]):
                list of semitones to transpose the data by
            strip_rest_edge_measures:
                whether or not to skip measures before the first and after the last melody note
        """
        
        self.data_dir = data_dir
        self.transpose = transpose
        self.strip_rest_edge_measures = strip_rest_edge_measures
        
        self.melody_sequence = torch.Tensor()
        self.harmony_sequence = torch.Tensor()
        
        for file in os.listdir(data_dir):
            with open(os.path.join(data_dir, file)) as f_in:
                file_text = f_in.read()
                
            lines = file_text.split("\n")
            lines = [line for line in lines if line != ""]
            
            song_has_melody = False
            for line in lines:
                if line.split()[0] != "12":
                    song_has_melody = True
                    break
                        
            if song_has_melody:
                if strip_rest_edge_measures:
                    while lines[0].split()[0] == "12":
                        lines.pop(0)

                    while lines[-1].split()[0] == "12":
                        lines.pop(-1)
                    
                song_melody = []
                song_harmony = []

                for line in lines:
                    melody_index, harmony_vector = line.split()
                    melody_index = int(melody_index)
                    song_melody.append([melody_index])
                    song_harmony.append([int(val) for val in harmony_vector])
                    
                song_melody = torch.tensor(song_melody, dtype=torch.float)
                song_harmony = torch.tensor(song_harmony, dtype=torch.float)
                
                self.melody_sequence = torch.cat((self.melody_sequence, song_melody))
                self.harmony_sequence = torch.cat((self.harmony_sequence, song_harmony))
                    
                if transpose:
                    for semitone_amount in transpose:
                        dot_index_bitmask = (song_melody != 12)
                        shifted_melody = copy.deepcopy(song_melody)
                        shifted_melody[dot_index_bitmask] = (song_melody[dot_index_bitmask] + semitone_amount) % 12
                        shifted_harmony = copy.deepcopy(song_harmony).roll(semitone_amount, dims=1)

                        self.melody_sequence = torch.cat((self.melody_sequence, shifted_melody))
                        self.harmony_sequence = torch.cat((self.harmony_sequence, shifted_harmony))
                    
            else:
                print("Skipping entry with no melody")

        self.melody_sequence = self.melody_sequence.view(1, -1)
        self.harmony_sequence = self.harmony_sequence.view(1, -1)
        
        self.melody_sequence = self.melody_sequence.view(48, -1, 1)
        self.harmony_sequence = self.harmony_sequence.view(48, -1, 12)

    def __len__(self):
        return ceil(self.melody_sequence.shape[1] / 64)
    
    def __getitem__(self, idx):
        melody, harmony = self.melody_sequence[:, idx*64:(idx + 1)*64], self.harmony_sequence[:, idx*64:(idx+1)*64]
        
        return {"melody": melody, "harmony": harmony}

In [3]:
training_data_fp = input("Input the directory where training files are stored: ")

Input the directory where training files are stored: C:\Users\Danie\PycharmProjects\ChordGenerator\data\training_data\training_dt_longnotes


In [6]:
model_path = input("Input the name of the file you would like to store the model in: ")

Input the name of the file you would like to store the model in: C:\Users\Danie\PycharmProjects\ChordGenerator\data\models\model3.pt


In [9]:
dataset = MelodyIndexToChordSequenceDataset(
    training_data_fp,
    transpose=[i for i in range(1, 12)],
    strip_rest_edge_measures=True
)

dataloader = DataLoader(dataset)

len(dataloader)

Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody


1122

In [15]:
positive_examples = torch.zeros([12])

for datum in dataloader:
    data_batch = datum["harmony"][0, ].to(torch.float).sum(1).sum(0)
    positive_examples += data_batch

negative_examples = (len(dataloader) * 48 * 64) - positive_examples

positive_weights = torch.div(negative_examples, positive_examples).flatten()

print(positive_weights)

tensor([2.9330, 2.9330, 2.9330, 2.9330, 2.9330, 2.9330, 2.9330, 2.9330, 2.9330,
        2.9330, 2.9330, 2.9330])


In [18]:
# Define recurrent prediction model
class LSTMGenerator(nn.Module):
    
    def __init__(self):
        super(LSTMGenerator, self).__init__()
        self.embedding = nn.Embedding(13, 100)
        self.lstm = nn.LSTM(input_size=100, hidden_size=256, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(256, 12)
        
    def forward(self, x, hidden_in):
        x = self.embedding(x)
        x = x.view(48, -1, 100)
        x, h_out = self.lstm(x, hidden_in)
        x = self.fc1(x)
        return x, h_out


net = LSTMGenerator()

print(net)

LSTMGenerator(
  (embedding): Embedding(13, 100)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=256, out_features=12, bias=True)
)


In [19]:
# Train recurrent model

criterion = nn.BCEWithLogitsLoss(weight=positive_weights)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5):
    
    hidden_out = (torch.randn(2, 48, 256),
                  torch.randn(2, 48, 256))
    
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data["melody"].to(dtype=torch.long)[0, ]
        labels = data["harmony"].to(dtype=torch.float)[0, ]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, hidden_out = net(inputs, hidden_out)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        hidden_out = (hidden_out[0].detach(), hidden_out[1].detach())

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')

[1,    50] loss: 2.011
[1,   100] loss: 1.976
[1,   150] loss: 1.943
[1,   200] loss: 1.914
[1,   250] loss: 1.885
[1,   300] loss: 1.852
[1,   350] loss: 1.822
[1,   400] loss: 1.799
[1,   450] loss: 1.772
[1,   500] loss: 1.745
[1,   550] loss: 1.724
[1,   600] loss: 1.709
[1,   650] loss: 1.698
[1,   700] loss: 1.689
[1,   750] loss: 1.687
[1,   800] loss: 1.686
[1,   850] loss: 1.682
[1,   900] loss: 1.674
[1,   950] loss: 1.669
[1,  1000] loss: 1.667
[1,  1050] loss: 1.661
[1,  1100] loss: 1.655
[2,    50] loss: 1.654
[2,   100] loss: 1.656
[2,   150] loss: 1.658
[2,   200] loss: 1.659
[2,   250] loss: 1.660
[2,   300] loss: 1.654
[2,   350] loss: 1.652
[2,   400] loss: 1.652
[2,   450] loss: 1.651
[2,   500] loss: 1.646
[2,   550] loss: 1.645
[2,   600] loss: 1.645
[2,   650] loss: 1.646
[2,   700] loss: 1.649
[2,   750] loss: 1.652
[2,   800] loss: 1.655
[2,   850] loss: 1.655
[2,   900] loss: 1.646
[2,   950] loss: 1.643
[2,  1000] loss: 1.642
[2,  1050] loss: 1.637
[2,  1100] 

In [20]:
torch.save(net.state_dict(), model_path)